# Learning about neural networks

## importing libs

In [1]:
import numpy as np

## Create an a activation function

here, we are going to be working with the sigmoid function

In [2]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

## Creating nodes

Create a single node which takes an input processes it weights and bias and runs and activation function over it

In [4]:
class Neuron:
    def __init__(self, weights, bias):
        self.weights = weights
        self.bias = bias
    
    def feedforward(self, x):
        total = np.dot(self.weights, x) + self.bias
        return sigmoid(total)

## Testing

lets check if it is actually working

In [6]:
x = np.array([2, 3])

n = Neuron([0, 1], 4)

n.feedforward(x)

0.9990889488055994

## Making a Neural Network
We combine multiple of these neurons (or nodes) to make a neural network

In [10]:
class NeuralNetwork:
    def __init__(self):
        weights = np.array([0, 1])
        bias = 0

        self.h1 = Neuron(weights, bias)
        self.h2 = Neuron(weights, bias)
        self.o1 = Neuron(weights, bias)

    def feedforward(self, x):
        out_h1 = self.h1.feedforward(x)
        out_h2 = self.h2.feedforward(x)

        new_x = np.array([out_h1, out_h2])

        return self.o1.feedforward(new_x)


## Lets check if this is running properly

In [11]:
nn = NeuralNetwork()

nn.feedforward(np.array([2, 3]))

0.7216325609518421

## Training a neural network

Training is done via a testing value. Here we are going to use the MSE(Mean Squared Error) to test against if the predicted values are close or not

In [12]:
def mse_loss(y_true, y_pred):
    return ((y_true - y_pred)**2).mean()

y_true = np.array([1, 0, 0, 1])
y_pred = np.array([0, 0, 0, 0])

mse_loss(y_true, y_pred)

0.5

## Minimize the error
The goal for use to train this neural network is to reduce this mse value

we use partial derivatives to propagate the errors back to the beginning to update the weights of the neural network

## Making a complete neural network

Name    |	Weight  (minus 135) |	Height (minus 66) |	Gender
Alice   |	-2                  |	-1                |	1
Bob     |    25                 |	6                 |	0
Charlie |	17                  |	4                 |	0
Diana   |	-15                 |	-6                |	1

In [13]:
def deriv_sigmoid(x):
    fx = sigmoid(x)

    return fx * (1 - fx)

In [14]:
class CompleteNeuralNetwork(NeuralNetwork):
    def __init__(self):
        self.w1 = np.random.normal()
        self.w2 = np.random.normal()
        self.w3 = np.random.normal()
        self.w4 = np.random.normal()
        self.w5 = np.random.normal()
        self.w6 = np.random.normal()

        self.b1 = np.random.normal()
        self.b2 = np.random.normal()
        self.b3 = np.random.normal()

    def feedforward(self, x):

        h1 = sigmoid(self.w1*x[0] + self.w2*x[1] + self.b1)
        h2 = sigmoid(self.w3*x[0] + self.w4*x[1] + self.b2)
        o1 = sigmoid(self.w5*h1 + self.w6*h2 + self.b3)
        return o1
        
    def train(self, data, all_y_trues):

        learn_rate = 0.1
        epochs = 1000

        for epoch in range(epochs):
            for x, y_true in zip(data, all_y_trues):
                sum_h1 = self.w1 * x[0] + self.w2 * x[1] + self.b1
                h1 = sigmoid(sum_h1)

                sum_h2 = self.w3 * x[0] + self.w4 * x[1] + self.b2
                h2 = sigmoid(sum_h2)

                sum_o1 = self.w5 * h1 + self.w6 * h2 + self.b3
                o1 = sigmoid(sum_o1)

                y_pred = o1

                d_L_d_ypred = -2 *(y_true - y_pred)
                
                #o1
                d_ypred_d_w5  = h1 * deriv_sigmoid(sum_o1)
                d_ypred_d_w6  = h2 * deriv_sigmoid(sum_o1)
                d_ypred_d_b3  = deriv_sigmoid(sum_o1)

                
                d_ypred_d_h1  = self.w5 * deriv_sigmoid(sum_o1)
                d_ypred_d_h2  = self.w6 * deriv_sigmoid(sum_o1)


                #h1
                d_h1_d_w1  = x[0] * deriv_sigmoid(sum_h1)
                d_h1_d_w2  = x[1] * deriv_sigmoid(sum_h1)
                d_h1_d_b1  = deriv_sigmoid(sum_h1)

                #h2
                d_h2_d_w3  = x[0] * deriv_sigmoid(sum_h2)
                d_h2_d_w4  = x[1] * deriv_sigmoid(sum_h2)
                d_h2_d_b2  = deriv_sigmoid(sum_h2)


                ## Calculate the update value
                # h1 updated
                self.w1 -= learn_rate * d_L_d_ypred * d_ypred_d_h1 * d_h1_d_w1
                self.w2 -= learn_rate * d_L_d_ypred * d_ypred_d_h1 * d_h1_d_w2
                self.b1 -= learn_rate * d_L_d_ypred * d_ypred_d_h1 * d_h1_d_b1
        
                # h1 updated
                self.w3 -= learn_rate * d_L_d_ypred * d_ypred_d_h2 * d_h2_d_w3
                self.w4 -= learn_rate * d_L_d_ypred * d_ypred_d_h2 * d_h2_d_w4
                self.b2 -= learn_rate * d_L_d_ypred * d_ypred_d_h2 * d_h2_d_b2

                # o1 updated
                self.w5 -= learn_rate * d_L_d_ypred * d_ypred_d_w5
                self.w6 -= learn_rate * d_L_d_ypred * d_ypred_d_w6
                self.b3 -= learn_rate * d_L_d_ypred * d_ypred_d_b3

            if  epoch % 10 == 0:
                y_preds = np.apply_along_axis(self.feedforward, 1, data)
                loss = mse_loss(all_y_trues, y_preds)
                print(f"epoch {epoch} loss: %.3f" % (loss))

### Entering dataset

In [15]:
data = np.array([
  [-2, -1],  # Alice
  [25, 6],   # Bob
  [17, 4],   # Charlie
  [-15, -6], # Diana
])
all_y_trues = np.array([
  1, # Alice
  0, # Bob
  0, # Charlie
  1, # Diana
])

In [16]:
network = CompleteNeuralNetwork()
network.train(data, all_y_trues)

epoch 0 loss: 0.240
epoch 10 loss: 0.137
epoch 20 loss: 0.095
epoch 30 loss: 0.071
epoch 40 loss: 0.056
epoch 50 loss: 0.045
epoch 60 loss: 0.037
epoch 70 loss: 0.031
epoch 80 loss: 0.027
epoch 90 loss: 0.024
epoch 100 loss: 0.021
epoch 110 loss: 0.019
epoch 120 loss: 0.017
epoch 130 loss: 0.015
epoch 140 loss: 0.014
epoch 150 loss: 0.013
epoch 160 loss: 0.012
epoch 170 loss: 0.011
epoch 180 loss: 0.010
epoch 190 loss: 0.010
epoch 200 loss: 0.009
epoch 210 loss: 0.009
epoch 220 loss: 0.008
epoch 230 loss: 0.008
epoch 240 loss: 0.007
epoch 250 loss: 0.007
epoch 260 loss: 0.007
epoch 270 loss: 0.006
epoch 280 loss: 0.006
epoch 290 loss: 0.006
epoch 300 loss: 0.006
epoch 310 loss: 0.006
epoch 320 loss: 0.005
epoch 330 loss: 0.005
epoch 340 loss: 0.005
epoch 350 loss: 0.005
epoch 360 loss: 0.005
epoch 370 loss: 0.004
epoch 380 loss: 0.004
epoch 390 loss: 0.004
epoch 400 loss: 0.004
epoch 410 loss: 0.004
epoch 420 loss: 0.004
epoch 430 loss: 0.004
epoch 440 loss: 0.004
epoch 450 loss: 0.004

## Lets predict

In [17]:
emily = np.array([-7, -3]) # 128 pounds, 63 inches
frank = np.array([20, 2])  # 155 pounds, 68 inches
print("Emily: %.3f" % network.feedforward(emily)) # 0.951 - F
print("Frank: %.3f" % network.feedforward(frank)) # 0.039 - M

Emily: 0.968
Frank: 0.039
